In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import regex

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('tokenize')

[nltk_data] Error loading tokenize: Package 'tokenize' not found in
[nltk_data]     index


False

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
text = "Three or four switches have stopped working, and one switch's LED light also stopped suddenly after exactly one year of buying it. It's important to note that the service center is only located in Mumbai, which means it may be difficult to get help if you're not in that area. Based on these issues, it is strongly not recommended to purchase this product.Think before buying. Only after one month of usage the T key has started giving issues, at some cases it takes 3 looooooong presses for the T o appear. For this review also I am trying o press hard on he key o make i work. But all he places you are no seeing the 'T' means, it didnt come even afer trying more than thrice. Invest in a better one if you are really looking for a long lasting keyboard. Having said that, the look and feel is amazing but wha's the need when he keys do not work.After getting to Keyboard I had a problem with LED which is not working and keys are getting stuck in between. while claiming warranty is very hard task. You have to courier and all."
words = nltk.word_tokenize(text)
tags = nltk.pos_tag(words)
print(tags)


[('Three', 'CD'), ('or', 'CC'), ('four', 'CD'), ('switches', 'NNS'), ('have', 'VBP'), ('stopped', 'VBN'), ('working', 'VBG'), (',', ','), ('and', 'CC'), ('one', 'CD'), ('switch', 'NN'), ("'s", 'POS'), ('LED', 'NNP'), ('light', 'NN'), ('also', 'RB'), ('stopped', 'VBD'), ('suddenly', 'RB'), ('after', 'IN'), ('exactly', 'RB'), ('one', 'CD'), ('year', 'NN'), ('of', 'IN'), ('buying', 'VBG'), ('it', 'PRP'), ('.', '.'), ('It', 'PRP'), ("'s", 'VBZ'), ('important', 'JJ'), ('to', 'TO'), ('note', 'VB'), ('that', 'IN'), ('the', 'DT'), ('service', 'NN'), ('center', 'NN'), ('is', 'VBZ'), ('only', 'RB'), ('located', 'VBN'), ('in', 'IN'), ('Mumbai', 'NNP'), (',', ','), ('which', 'WDT'), ('means', 'VBZ'), ('it', 'PRP'), ('may', 'MD'), ('be', 'VB'), ('difficult', 'JJ'), ('to', 'TO'), ('get', 'VB'), ('help', 'VB'), ('if', 'IN'), ('you', 'PRP'), ("'re", 'VBP'), ('not', 'RB'), ('in', 'IN'), ('that', 'DT'), ('area', 'NN'), ('.', '.'), ('Based', 'VBN'), ('on', 'IN'), ('these', 'DT'), ('issues', 'NNS'), (',',

In [ ]:
from nltk import bigrams
bigrams = list(bigrams(words))
print(bigrams)

[('Three', 'or'), ('or', 'four'), ('four', 'switches'), ('switches', 'have'), ('have', 'stopped'), ('stopped', 'working'), ('working', ','), (',', 'and'), ('and', 'one'), ('one', 'switch'), ('switch', "'s"), ("'s", 'LED'), ('LED', 'light'), ('light', 'also'), ('also', 'stopped'), ('stopped', 'suddenly'), ('suddenly', 'after'), ('after', 'exactly'), ('exactly', 'one'), ('one', 'year'), ('year', 'of'), ('of', 'buying'), ('buying', 'it'), ('it', '.'), ('.', 'It'), ('It', "'s"), ("'s", 'important'), ('important', 'to'), ('to', 'note'), ('note', 'that'), ('that', 'the'), ('the', 'service'), ('service', 'center'), ('center', 'is'), ('is', 'only'), ('only', 'located'), ('located', 'in'), ('in', 'Mumbai'), ('Mumbai', ','), (',', 'which'), ('which', 'means'), ('means', 'it'), ('it', 'may'), ('may', 'be'), ('be', 'difficult'), ('difficult', 'to'), ('to', 'get'), ('get', 'help'), ('help', 'if'), ('if', 'you'), ('you', "'re"), ("'re", 'not'), ('not', 'in'), ('in', 'that'), ('that', 'area'), ('area

JJ-NN/NNS
RB/RBR/RBS-JJ

In [ ]:
def filter_bigrams(tags):
    filtered_bigrams = []
    for i in range(len(tags) - 1):
        first_word, first_tag = tags[i]
        second_word, second_tag = tags[i + 1]

        if (first_tag.startswith('JJ') and second_tag in ['NN', 'NNS']) \
            or (first_tag in ['RB', 'RBR', 'RBS'] and second_tag.startswith('JJ') and not (second_tag == 'NN' or second_tag == 'NNS')) \
            or (first_tag.startswith('JJ') and second_tag.startswith('JJ') and not (second_tag == 'NN' or second_tag == 'NNS')) \
            or ((first_tag == 'NN' or first_tag == 'NNS') and second_tag.startswith('JJ') and not (second_tag == 'NN' or second_tag == 'NNS')) \
            or (first_tag in ['RB', 'RBR', 'RBS'] and second_tag.startswith('VB') and second_tag != 'NNS'):
            filtered_bigrams.append((first_word, second_word))
    return filtered_bigrams

filtered_bigrams = filter_bigrams(tags)

for bigram in filtered_bigrams:
  print(bigram)

('also', 'stopped')
('only', 'located')
('not', 'recommended')
('looooooong', 'presses')
('o', 'press')
('key', 'o')
('o', 'make')
('even', 'afer')
('really', 'looking')
('long', 'lasting')
('lasting', 'keyboard')
('not', 'work.After')
('not', 'working')
('very', 'hard')
('hard', 'task')


In [ ]:
from nltk.probability import FreqDist
from math import log

def compute_pmi(filtered_bigrams, words):
  pmi_dict = {}
  word_freq = FreqDist(words)
  bigram_freq = FreqDist(filtered_bigrams)
  for bigram in filtered_bigrams:
    word1, word2 = bigram
    pmi = log((bigram_freq[bigram] * len(words)) / (word_freq[word1] * word_freq[word2]))
    pmi_dict[bigram] = pmi
  return pmi_dict

pmi_dict = compute_pmi(filtered_bigrams, words)

for bigram, pmi in pmi_dict.items():
  print(f"{bigram}: {pmi}")


('also', 'stopped'): 3.979681653901961
('only', 'located'): 5.365976015021851
('not', 'recommended'): 3.979681653901961
('looooooong', 'presses'): 5.365976015021851
('o', 'press'): 4.267363726353742
('key', 'o'): 3.574216545793796
('o', 'make'): 4.267363726353742
('even', 'afer'): 5.365976015021851
('really', 'looking'): 5.365976015021851
('long', 'lasting'): 5.365976015021851
('lasting', 'keyboard'): 5.365976015021851
('not', 'work.After'): 3.979681653901961
('not', 'working'): 3.2865344733420154
('very', 'hard'): 4.672828834461906
('hard', 'task'): 4.672828834461906


In [ ]:
# Define positive and negative words
positive_words = ["good", "great", "excellent", "amazing", "wonderful"]
negative_words = ["not", "stopped", "difficult", "horrible", "Problem"]

# Filter bigrams based on sentiment
positive_bigrams = [bigram for bigram in filtered_bigrams if bigram[0] in positive_words or bigram[1] in positive_words]
negative_bigrams = [bigram for bigram in filtered_bigrams if bigram[0] in negative_words or bigram[1] in negative_words]

# Compute PMI for positive and negative bigrams
positive_pmi_dict = compute_pmi(positive_bigrams, words)
negative_pmi_dict = compute_pmi(negative_bigrams, words)

# Print PMI for positive and negative bigrams
print("Positive Bigrams:")
for bigram, pmi in positive_pmi_dict.items():
  print(f"{bigram}: {pmi}")

print("\nNegative Bigrams:")
for bigram, pmi in negative_pmi_dict.items():
  print(f"{bigram}: {pmi}")


Positive Bigrams:

Negative Bigrams:
('also', 'stopped'): 3.979681653901961
('not', 'recommended'): 3.979681653901961
('not', 'work.After'): 3.979681653901961
('not', 'working'): 3.2865344733420154
